<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_vision_qwen_vl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transformers qwen2-vl

In [ ]:
!nvidia-smi && lscpu

In [ ]:
!pip show torch torchvision torchaudio transformers autoawq

In [ ]:
!pip install -q -U autoawq torch torchvision torchaudio git+https://github.com/huggingface/transformers

In [ ]:
!pip show torch torchvision torchaudio transformers autoawq

In [ ]:
!pip install -q qwen-vl-utils

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from transformers import TextIteratorStreamer
import torch

from qwen_vl_utils import process_vision_info
from time import perf_counter

model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-Int8"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-Int4"
model_ckpt_name = "Qwen/Qwen2-VL-7B-Instruct-AWQ"

model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-Int8"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-Int4"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct-AWQ"
model_ckpt_name = "Qwen/Qwen2-VL-2B-Instruct"


# default: Load the model on the available device(s)
#model = Qwen2VLForConditionalGeneration.from_pretrained(
#    model_ckpt_name, torch_dtype=torch.float16, device_map="cuda"
#)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# see detail: https://huggingface.co/docs/transformers/perf_infer_gpu_one
model = Qwen2VLForConditionalGeneration.from_pretrained(
     model_ckpt_name,
     torch_dtype="auto",
     #attn_implementation="sdpa", #sdpa or flash_attention_2, no eager
     #attn_implementation="flash_attention_2", #sdpa or flash_attention_2, no eager
     device_map="cuda",
)

# default processer
#processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(model_ckpt_name, min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "描述下图片内容"},
        ],
    }
]
times=[]
start_time = perf_counter()
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)


# Inference: Generation of the output with TextIteratorStreamer
streamer = TextIteratorStreamer(processor, skip_prompt=True, skip_special_tokens=True)
# Use Thread to run generation in background
# Otherwise, the process is blocked until generation is complete
# and no streaming effect can be observed.
from threading import Thread
generation_kwargs = dict(**inputs, streamer=streamer, max_new_tokens=128)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

generated_text = ""
for new_text in streamer:
    #print(new_text)
    times.append(perf_counter() - start_time)
    generated_text += new_text
    start_time = perf_counter()
print(generated_text)

print("times",times)
print("total cost:",sum(times))

# achatbot vision lm qwen

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot

In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.8.6-py3-none-any.whl[tensorrt,daily_room_audio_stream,livekit,livekit-api,sense_voice_asr,deepgram_asr_processor,llm_transformers_manual_vision,einops,llm_transformers_manual_vision_img_janus,tts_edge,openai,queue]"

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall --quiet --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download Qwen/Qwen2-VL-7B-Instruct --quiet --local-dir ./models/Qwen/Qwen2-VL-7B-Instruct --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download Qwen/Qwen2-VL-2B-Instruct --quiet --local-dir ./models/Qwen/Qwen2-VL-2B-Instruct --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download unsloth/Llama-3.2-11B-Vision-Instruct --quiet --local-dir ./models/unsloth/Llama-3.2-11B-Vision-Instruct --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download allenai/Molmo-7B-D-0924 --quiet --local-dir ./models/allenai/Molmo-7B-D-0924 --local-dir-use-symlinks False

In [ ]:
# cpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

In [ ]:
# gpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" LLM_DEVICE=cuda \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

In [ ]:
# gpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" LLM_DEVICE=cuda \
    VIDEO_FILE="/content/capture3_2024-09-15_20-10-27.mp4" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

In [ ]:
# gpu
!LLM_MODEL_NAME_OR_PATH="./models/Qwen/Qwen2-VL-2B-Instruct" LLM_DEVICE=cuda LLM_CHAT_HISTORY_SIZE=0 \
    VIDEO_FILE="/content/capture3_2024-09-15_20-10-27.mp4" \
    python -m unittest test.core.llm.test_transformers_v_qwen.TestTransformersVQwen.test_chat_completion_prompts

In [ ]:
!LLM_TAG=llm_transformers_manual_vision_molmo \
    LLM_MODEL_NAME_OR_PATH=allenai/Molmo-7B-D-0924 \
    python -m unittest test.core.llm.test_transformers_v_llama.TestTransformersVLlama.test_chat_completion_prompts

In [ ]:
!LLM_TAG=llm_transformers_manual_vision_molmo \
    LLM_MODEL_NAME_OR_PATH=allenai/Molmo-7B-D-0924 \
    LLM_CHAT_HISTORY_SIZE=0 \
    python -m unittest test.integration.processors.test_vision_processor.TestVisionProcessor

In [3]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')


run bot task woker with bot.json, e.g.: dummy_bot.json

- use daily/livekit room stream, u can click bot joined the room url, to start chat with bot with audio and camera stream,
  - [daily](https://www.daily.co/) need DAILY_API_KEY
  - [livekit](https://livekit.io/) need project url LIVEKIT_URL, LIVEKIT_API_KEY, LIVEKIT_API_SECRET



In [4]:
!cat /content/daily_describe_transformers_vision_bot.json

{
  "chat_bot_name": "DailyDescribeVisionBot",
  "room_name": "chat-bot",
  "room_url": "",
  "token": "",
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "transformers_manual_vision_qwen",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag":"llm_transformers_manual_vision_qwen",
      "args":{
        "lm_device":"cuda",
        "lm_model_name_or_path":"./models/Qwen/Qwen2-VL-2B-Instruct",
        "chat_history_size": 0,
        "init_chat_prompt":"请用中文交流",
        "model_type":"chat_completion"
      },
      "language": "zh"
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",
        "language": "zh",
        

In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY python -m src.cmd.bots.main -f /content/daily_describe_transformers_vision_bot.json

In [ ]:
!cat /content/livekit_describe_transformers_vision_bot.json

{
  "chat_bot_name": "LivekitDescribeVisionBot",
  "room_name": "chat-bot",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "livekit_room",
    "args": {
      "bot_name": "LivekitDescribeVisionBot",
      "is_common_session": false
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "transformers_manual_vision_qwen",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "llm_transformers_manual_vision_qwen",
      "args": {
        "lm_device": "cuda",
        "lm_model_name_or_path": "./models/Qwen/Qwen2-VL-2B-Instruct",
        "chat_history_size": 0,
        "init_chat_prompt": "请用中文交流",
        "model_type": "chat_completion"
      },
    

In [ ]:
!LIVEKIT_URL=$LIVEKIT_URL LIVEKIT_API_KEY=$LIVEKIT_API_KEY LIVEKIT_API_SECRET=$LIVEKIT_API_SECRET \
  python -m src.cmd.bots.main -f /content/livekit_describe_transformers_vision_bot.json

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')

In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json